Imports

In [21]:
from riotwatcher import LolWatcher, ApiError, RiotWatcher
from dotenv import load_dotenv

import json
import pandas as pd
import time
import os

Access Data and Keys

In [22]:
load_dotenv()

api_key = os.getenv("RIOT_API_KEY")
region = os.getenv("REGION")
platform = os.getenv("PLATFORM")

lol_watcher = LolWatcher(api_key)
riot_watcher = RiotWatcher(api_key)

all_matches = {}
match_data = []
summoner_list = []
results = []

RiotWatcher & Match data

In [23]:
with open('summoners_euw.txt', 'r') as file:
    summoner_names = file.readlines()

for names in summoner_names:
    user_name = names.strip("#")
    summoner_list.append(user_name)

split_summoner_list = []
for item in summoner_list:
    if '#' in item:
        parts = item.split('#')
        split_parts = [part.strip() for part in parts]
        split_summoner_list.append(split_parts)
    else:
        split_summoner_list.append([item.strip()])

for summoners in range(len(split_summoner_list)):
    account = riot_watcher.account.by_riot_id(platform, split_summoner_list[summoners][0], split_summoner_list[summoners][1])
    match_ids = lol_watcher.match.matchlist_by_puuid(platform, account['puuid'], count=1)

# TODO: SummonerName is parametrized (maybe for-loop for everyone)

In [24]:
for match_id in match_ids:
    match = lol_watcher.match.by_id(platform, match_id)
    match_data.append(match)
    time.sleep(1.5)

all_matches[account['gameName']] = match_data
print(f"Retrieved {len(match_data)} matches for {account['gameName']}\n")

#print(all_matches)

with open('all_matches.json', 'w') as file:
    json.dump(all_matches, file, indent=2)

with open('participants.json', 'w') as file:
    json.dump(all_matches['Agurin'][0]["metadata"]["participants"][1:], file, indent=2)

Retrieved 1 matches for Agurin



Fetching More Accounts

In [37]:
with open('participants.json', 'r') as file:
    summoner_idx = json.load(file) # TODO: Not all participants work for now

summoner_idx = sorted(set(p for p in summoner_idx if isinstance(p, str) and len(p) > 20))

for idx, puuid in enumerate(summoner_idx, 1): #enumerate is just für a counter for us to know how many summoners we have
    try:
        account = riot_watcher.account.by_puuid(platform, puuid)
        results.append({
            "ID": idx,
            "gameName": account.get("gameName"),
            "tagLine": account.get("tagLine"),
            "puuid": puuid
        })
        time.sleep(1.2)  # Respect Riot API rate limits
    except Exception as e:
        results.append({
            "ID": idx,
            "error": str(e),
            "puuid": puuid
        })

with open('puuid_accounts.json', "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2)

# TODO: Finding more participants without duplicates